In [1]:
from datetime import datetime
from re import findall
from Crypto.Hash import SHA
from requests import Session
from bs4 import BeautifulSoup as bs

In [2]:
# Usuario y contraseña de la pagina web de monitoreo de temperaturas
user="mexsat"
pswd="mexsat"

# Se utiliza algoritmo SHA256 para hasheo de contraseña
h=SHA.new()
h.update(pswd.encode())
pswd_sha=h.hexdigest()

# La dirección base de la pagina de monitoreo de temperaturas
base_url="http://192.168.169.3:81/"

# Se establece la información a enviar para login en pagina de monitoreo
# de temperaturas
login_data = {"uid_in":"mexsat", "pid_in":"", "hex_in":pswd_sha}

In [3]:
docs = []
# Se inicia sesion de web scraping
with Session() as ses:
    try:
        # Se envía peticion a la página web agregando el header inicial y
        # la información de login
        response = ses.post(base_url + ".html?get=${|mexsat|}", login_data)
        # A la respuesta de esta petición se le decodifica con un parser
        # para html
        doc      = bs(response.text, 'html.parser')
        docs.append(doc)
        # Se obtiene el primer bloque de la página identificado como script
        # lo que nos indica que es Javascript
        script   = doc.find_all("script")[0].contents[0]
        
        # Dentro de este script se encuentra el identificador de la pagina,
        # usuario y sessionID necesarios para la siguiente petición
        pag, user, sessionID = findall(".*\$(\w*)\|(\w*)\|(\w*)';", script)[0]
        
        # Se construye header de respuesta para acceder a página principal
        response_header = f".html?get=${pag}|{user}|{sessionID}"
        # Se envía petición de página principal
        response  = ses.get(base_url + response_header)
        # Se decodifica respuesta
        doc       = bs(response.text, 'html.parser')
        docs.append(doc)
        # Se obtiene bloque de script
        script    = doc.find_all("script")[0].contents[0]
        # Se obtiene sessionID
        sessionID = findall('.*var sessionID = "([^\n\r]*)";', script)[0]
        # Se define un arreglo con los headers para las peticiones y el
        # numero de tag en donde se localiza la temperatura en cada página
        param = [[f".html?get=$AHU01", "0"],
                 [f".html?get=$AHU02", "0"],
                 [f".html?get=$AHU03", "0"],
                 [f".html?get=$AHU04", "0"],
                 [f".html?get=$AHU05", "0"],
                 [f".html?get=$AHU10", "2"],
                 [f".html?get=$AHU11", "37"],
                 [f".html?get=$CHILLER", "0"],
                 [f".html?get=$CH02", "0"]]

        # Se inicializa una variable para guardar las temperaturas
        temps = []
        # Para cada una de las páginas a revisar
        for i in param:
            # Se construye header de respuesta
            response_header = i[0] + f"|{user}|{sessionID}"
            # Se envía petición
            response = ses.get(base_url + response_header)
            # Se decodifica respuesta
            doc = bs(response.text, 'html.parser')
            docs.append(doc)
            # Se obtiene bloque de script
            script = doc.find_all("script")[0].contents[0]
            # Se obtiene sessionID
            sessionID = findall('.*var sessionID = "([^\n\r]*)";', script)[0]
            # Se obtiene el cuerpo de la página
            cont = doc.find_all("body")[0]
            # Se construye el patron de busqueda para la temperatura
            patron  = 'id="tag'
            patron +=  i[1]
            patron += '" nowrap="" style="color: #0040e0" width="100%">[\n\r]\s*(\d{2})'
            # Se aplica la busqueda
            #t = findall(patron, cont.prettify())[0]
            #temps.append(int(t))
            
    # En caso de que ocurra algun error, se apaga la bandera de obtencion
    # de temperaturas exitosa
    except Exception:
        current = False

In [4]:
for i, doc in enumerate(docs):
    with open("./pags/pag" + str(i) + ".html", "w") as file:
        file.write(doc.prettify())

In [5]:
test_str = '<img alt="Active Value 24" id="tag0" src="images/LEDs/green_on.gif"/>'

In [36]:
findall('<img .*"tag0".*src="(.*)"/>', docs[1].prettify())[0] in ['images/LEDs/green_on.gif', 'images/LEDs/LED_GRN_ON.gif']

True

In [38]:
findall('<img .*"tag1".*src="(.*)"/>', docs[1].prettify())[0] in ['images/LEDs/green_on.gif', 'images/LEDs/LED_GRN_ON.gif']

True

In [39]:
findall('<img .*"tag2".*src="(.*)"/>', docs[1].prettify())[0] in ['images/LEDs/green_on.gif', 'images/LEDs/LED_GRN_ON.gif']

True

In [49]:
a=1

In [50]:
f'pizza{a:02}'

'pizza01'

In [51]:
with open("conf_matplotlib.py", "r") as file:
    prueba = file.read()

In [52]:
prueba

'def conf_matplotlib_oscuro():\n    from matplotlib.pyplot import style, rcParams\n\n    style.use("dark_background")\n\n    rcParams["figure.facecolor"] = (43/256, 43/256, 43/256)\n    rcParams["axes.facecolor"]   = (43/256, 43/256, 43/256)\n    rcParams["legend.facecolor"] = (43/256, 43/256, 43/256)\n    \n    rcParams["legend.edgecolor"] = (43*4/256, 43*4/256, 43*4/256, 0.5)\n    rcParams["axes.edgecolor"]   = (43*3/256, 43*3/256, 43*3/256, 0.5)\n    \n    rcParams["axes.grid"]        = True\n    \n    rcParams["grid.color"]       = (43*4/256, 43*4/256, 43*4/256, 0.5)\n    \n    rcParams["xtick.color"]      = (43*4/256, 43*4/256, 43*4/256, 0.8)\n    rcParams["ytick.color"]      = (43*4/256, 43*4/256, 43*4/256, 0.8)\n    \n    rcParams["lines.linewidth"]  = 5\n    rcParams["grid.linewidth"]   = 0.5\n    \ndef conf_matplotlib_claro():\n    from matplotlib.pyplot import style, rcParams\n\n    style.use("ggplot")\n\n    #rcParams["figure.facecolor"] = (43/256, 43/256, 43/256)\n    #rcPa